In [5]:
import time
import threading
import importlib
import logging
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(threadName)s %(message)s')

import random
import datetime

def worker():
    time.sleep(random.randint(1, 6))

def boss():
    start = datetime.datetime.now()
    logging.info('worker exit {}'.format(datetime.datetime.now() - start))

In [6]:
# 5 - worker
# 1 - boss
# 1个 worker 退出的时候 boss 要计算他sleep 了 多久


In [7]:
event = threading.Event()

In [8]:
event.set()

In [9]:
event.wait()

True

In [17]:
def worker(event: threading.Event):
    s = random.randint(1, 6)
    time.sleep(s)
    event.set()
    logging.info('sleep {}'.format(s))

def boss(event: threading.Event):
#     logging.info('boss enter the room')
    start = datetime.datetime.now()
    event.wait()
    logging.info('worker exit {}'.format(datetime.datetime.now() - start))
    
def start():
    event = threading.Event()
    b = threading.Thread(target=boss, args=(event, ), name='boss')
    b.start()
    for x in range(5):
        threading.Thread(target=worker, args=(event, ), name='worker-{}'.format(x)).start()

In [18]:
start()

2017-11-11 14:27:36,793 INFO worker-1 sleep 1
2017-11-11 14:27:36,794 INFO boss worker exit 0:00:01.003339
2017-11-11 14:27:37,792 INFO worker-4 sleep 2
2017-11-11 14:27:37,792 INFO worker-2 sleep 2
2017-11-11 14:27:38,794 INFO worker-3 sleep 3
2017-11-11 14:27:41,795 INFO worker-0 sleep 6


In [19]:
def worker(event: threading.Event):
    s = random.randint(1, 6)
#     time.sleep(s)
    event.wait(s)
    event.set()
    logging.info('sleep {}'.format(s))

def boss(event: threading.Event):
#     logging.info('boss enter the room')
    start = datetime.datetime.now()
    event.wait()
    logging.info('worker exit {}'.format(datetime.datetime.now() - start))
    
def start():
    event = threading.Event()
    b = threading.Thread(target=boss, args=(event, ), name='boss')
    b.start()
    for x in range(5):
        threading.Thread(target=worker, args=(event, ), name='worker-{}'.format(x)).start()

In [20]:
start()

2017-11-11 14:30:19,419 INFO worker-4 sleep 1
2017-11-11 14:30:19,420 INFO worker-1 sleep 3
2017-11-11 14:30:19,420 INFO boss worker exit 0:00:01.005523
2017-11-11 14:30:19,420 INFO worker-0 sleep 5
2017-11-11 14:30:19,420 INFO worker-2 sleep 4
2017-11-11 14:30:19,420 INFO worker-3 sleep 2


通常用于 某个线程需要等待其他线程处理完成 某些动作之后  才能启动

In [21]:
event

In [23]:
event = threading.Event()

In [26]:
event.wait(1)

False

In [27]:
event.set()

In [28]:
event.wait(1)

True

**event 在没有被set 过的时候， 返回的是 False，一旦被set 过了，返回的 就都是True 了**

In [25]:
def worker(event: threading.Event):
    while True:
        logging.info('run run run')
        time.sleep(3)

In [29]:
def worker(event: threading.Event):
    while not event.wait(3): # 这是一个技巧
        logging.info('run run run')

In [30]:
event = threading.Event()

In [31]:
threading.Thread(target=worker, args=(event, ), name='printer').start()

2017-11-11 14:40:23,625 INFO printer run run run
2017-11-11 14:40:26,627 INFO printer run run run
2017-11-11 14:40:29,629 INFO printer run run run
2017-11-11 14:40:32,634 INFO printer run run run


In [32]:
event.set()

可以通过这种技巧 来完成 对线程退出的 控制条件

这也是 为什么我说线程的退出条件 写在了我们的 逻辑里

In [33]:
event.is_set()

True

In [34]:
event.clear()

In [36]:
event.is_set() # 默认情况下，返回的都是False

False

In [37]:
event.set()

In [38]:
event.is_set()

True

In [39]:
def worker(event):
    while not event.is_set():
        pass

event 无论在哪一个线程里面 wait，只要在其他任意地方（同一个进程下）set，他就退出了

event.wait(3) 和 sleep(3) 有什么区别？
- wait 会主动让出时间片
- sleep 不会主动让出时间片

In [43]:
class Timer:
    def __init__(self, interval, function, *args, **kwargs):
        self.interval = interval
        self.function = function
        self.args = args
        self.kwargs = kwargs
        self.event = threading.Event()
        self.thread = threading.Thread(target=self.__target)
        
    def __target(self):
        if not self.event.wait(self.interval):
            self.function(*self.args, **self.kwargs)
            
    def start(self):
        self.thread.start()
        
    def cancel(self):
        self.event.set()

In [44]:
def worker():
    logging.info('run')

In [48]:
t = Timer(interval=5, function=worker)

In [49]:
t.start()

In [50]:
t.cancel()

## Lock

In [51]:
class Counter:
    def __init__(self):
        self.__val = 0
        
    @property
    def value(self):
        return self.__val
    
    def inc(self):
        self.__val += 1
        
    def dec(self):
        self.__val -= 1

In [69]:
counter = Counter()

In [53]:
counter.inc()

In [54]:
counter.value

1

In [55]:
counter.dec()

In [56]:
counter.value

0

In [66]:
def fn():
    if random.choice([1, -1]) > 0:
        logging.info('inc')
        counter.inc()
    else:
        logging.info('dec')
        counter.dec()

In [70]:
for x in range(10):
    threading.Thread(target=fn).start()

2017-11-11 15:19:06,555 INFO Thread-236 dec
2017-11-11 15:19:06,555 INFO Thread-237 inc
2017-11-11 15:19:06,556 INFO Thread-238 dec
2017-11-11 15:19:06,556 INFO Thread-239 dec
2017-11-11 15:19:06,557 INFO Thread-240 inc
2017-11-11 15:19:06,558 INFO Thread-241 inc
2017-11-11 15:19:06,559 INFO Thread-242 inc
2017-11-11 15:19:06,560 INFO Thread-243 inc
2017-11-11 15:19:06,561 INFO Thread-244 inc
2017-11-11 15:19:06,561 INFO Thread-245 inc


In [71]:
counter.value

4

In [72]:
lock = threading.Lock()

In [73]:
lock.acquire()

True

In [74]:
1 + 2

3

In [75]:
lock.acquire()

KeyboardInterrupt: 

In [78]:
lock.release()

In [77]:
lock.acquire()

True

In [80]:
class Counter:
    def __init__(self):
        self.__val = 0
        self._lock = threading.Lock()
        
    @property
    def value(self):
        return self.__val
    
    def inc(self):
        try:
            self._lock.acquire()
            self.__val += 1
        finally:
            self._lock.release()
        
    def dec(self):
        try:
            self._lock.acquire()
            self.__val -= 1
        finally:
            self._lock.release()

**凡是用锁的地方，必须在finally 里面 进行release，否则就会产生死锁**

In [83]:
class Counter:
    def __init__(self):
        self.__val = 0
        self._lock = threading.Lock()
        
    @property
    def value(self):
        with self._lock:
            return self.__val
    
    def inc(self):
        with self._lock:
            self.__val += 1
        
    def dec(self):
        with self._lock:
            self.__val -= 1

In [84]:
counter = Counter()

In [85]:
def fn():
    if random.choice([1, -1]) > 0:
        logging.info('inc')
        counter.inc()
    else:
        logging.info('dec')
        counter.dec()

In [86]:
for x in range(10):
    threading.Thread(target=fn).start()

2017-11-11 15:30:34,784 INFO Thread-246 dec
2017-11-11 15:30:34,785 INFO Thread-247 dec
2017-11-11 15:30:34,785 INFO Thread-248 inc
2017-11-11 15:30:34,786 INFO Thread-249 inc
2017-11-11 15:30:34,786 INFO Thread-250 inc
2017-11-11 15:30:34,787 INFO Thread-251 dec
2017-11-11 15:30:34,787 INFO Thread-252 dec
2017-11-11 15:30:34,787 INFO Thread-253 dec
2017-11-11 15:30:34,788 INFO Thread-254 inc
2017-11-11 15:30:34,788 INFO Thread-255 dec


In [87]:
lock

<unlocked _thread.lock object at 0x10adb7e90>

In [88]:
lock.acquire(blocking=False)

True

In [89]:
lock.acquire(blocking=False)

False

In [90]:
lock.release()

In [91]:
lock.acquire(timeout=5)

True

In [92]:
lock.acquire(timeout=5)

False

In [96]:
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):

            logging.info(task.name)

class Task:
    def __init__(self, name):
        self.name = name
        self.lock = threading.Lock()

In [97]:
tasks = [Task(x) for x in range(10)]

In [98]:
for x in range(5):
    threading.Thread(target=worker, args=(tasks, ), name='worker-{}'.format(x)).start()

2017-11-11 16:01:37,251 INFO worker-0 0
2017-11-11 16:01:37,252 INFO worker-1 1
2017-11-11 16:01:37,252 INFO worker-2 2
2017-11-11 16:01:37,253 INFO worker-3 3
2017-11-11 16:01:37,253 INFO worker-0 4
2017-11-11 16:01:37,253 INFO worker-4 5
2017-11-11 16:01:37,254 INFO worker-1 6
2017-11-11 16:01:37,255 INFO worker-2 7
2017-11-11 16:01:37,257 INFO worker-3 8
2017-11-11 16:01:37,258 INFO worker-0 9


### 可重入锁

In [99]:
rlock = threading.RLock()

In [100]:
rlock.acquire()

True

In [101]:
rlock.acquire()

True

In [102]:
rlock.acquire()

True

In [103]:
rlock.acquire(blocking=False)

True

In [104]:
rlock.release()

In [109]:
rlock.release()

RuntimeError: cannot release un-acquired lock

### Condition

生产者消费者模型

In [110]:
class Dispatcher:
    def __init__(self):
        self.data = None
        
    def consumer(self):
        logging.info(self.data)
        
    def producer(self):
        data = random.randint(0, 100)
        logging.info(data)
        self.data = data

In [111]:
d = Dispatcher()

In [117]:
p = threading.Thread(target=d.producer, name='producer')

In [118]:
c = threading.Thread(target=d.consumer, name='consumer')

In [119]:
p.start()

2017-11-11 16:39:57,351 INFO producer 96


In [120]:
c.start()

2017-11-11 16:39:59,017 INFO consumer 96


In [138]:
class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        
    def consumer(self):
        while not self.event.wait(1):
            logging.info(self.data)
        
    def producer(self):
        for _ in range(10):
            data = random.randint(0, 100)
            logging.info(data)
            self.data = data
            self.event.wait(1)
        self.event.set()

In [142]:
d = Dispatcher()
p = threading.Thread(target=d.producer, name='producer')
c = threading.Thread(target=d.consumer, name='consumer')

In [144]:
p.start()

2017-11-11 16:49:23,784 INFO producer 32
2017-11-11 16:49:24,653 INFO consumer 32
2017-11-11 16:49:24,791 INFO producer 21
2017-11-11 16:49:25,660 INFO consumer 21
2017-11-11 16:49:25,794 INFO producer 85
2017-11-11 16:49:26,665 INFO consumer 85
2017-11-11 16:49:26,800 INFO producer 61
2017-11-11 16:49:27,667 INFO consumer 61
2017-11-11 16:49:27,804 INFO producer 42
2017-11-11 16:49:28,670 INFO consumer 42
2017-11-11 16:49:28,810 INFO producer 14
2017-11-11 16:49:29,672 INFO consumer 14
2017-11-11 16:49:29,815 INFO producer 75
2017-11-11 16:49:30,678 INFO consumer 75
2017-11-11 16:49:30,820 INFO producer 9


In [143]:
c.start()

2017-11-11 16:49:21,643 INFO consumer None
2017-11-11 16:49:22,645 INFO consumer None
2017-11-11 16:49:23,649 INFO consumer None


In [161]:
class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        self.cond = threading.Condition()
        
    def consumer(self):
        while not self.event.is_set():
            with self.cond:
                self.cond.wait()
                logging.info(self.data)
        
    def producer(self):
        for _ in range(10):
            data = random.randint(0, 100)
            logging.info(data)
            self.data = data
            with self.cond:
                self.cond.notifyAll()
            self.event.wait(1)
        self.event.set()

In [162]:
d = Dispatcher()
p = threading.Thread(target=d.producer, name='producer')
c = threading.Thread(target=d.consumer, name='consumer')

In [155]:
c.start()

In [156]:
p.start()

2017-11-11 16:53:29,101 INFO producer 64
2017-11-11 16:53:29,104 INFO consumer 64
2017-11-11 16:53:30,106 INFO producer 30
2017-11-11 16:53:30,107 INFO consumer 30
2017-11-11 16:53:31,112 INFO producer 58
2017-11-11 16:53:31,114 INFO consumer 58
2017-11-11 16:53:32,116 INFO producer 95
2017-11-11 16:53:32,117 INFO consumer 95
2017-11-11 16:53:33,121 INFO producer 36
2017-11-11 16:53:33,122 INFO consumer 36
2017-11-11 16:53:34,126 INFO producer 52
2017-11-11 16:53:34,127 INFO consumer 52


In [163]:
for x in range(4):
    threading.Thread(target=d.consumer, name='consumer-{}'.format(x)).start()

In [159]:
p = threading.Thread(target=d.producer, name='producer')

In [164]:
p.start()

2017-11-11 16:56:38,077 INFO producer 91
2017-11-11 16:56:38,079 INFO consumer-0 91
2017-11-11 16:56:38,080 INFO consumer-2 91
2017-11-11 16:56:38,081 INFO consumer-1 91
2017-11-11 16:56:38,082 INFO consumer-3 91
2017-11-11 16:56:39,082 INFO producer 69
2017-11-11 16:56:39,083 INFO consumer-0 69
2017-11-11 16:56:39,085 INFO consumer-1 69
2017-11-11 16:56:39,086 INFO consumer-2 69
2017-11-11 16:56:39,088 INFO consumer-3 69
2017-11-11 16:56:40,086 INFO producer 15
2017-11-11 16:56:40,087 INFO consumer-0 15
2017-11-11 16:56:40,089 INFO consumer-3 15
2017-11-11 16:56:40,090 INFO consumer-2 15
2017-11-11 16:56:40,091 INFO consumer-1 15
2017-11-11 16:56:41,087 INFO producer 20
2017-11-11 16:56:41,089 INFO consumer-3 20
2017-11-11 16:56:41,090 INFO consumer-1 20
2017-11-11 16:56:41,091 INFO consumer-2 20
2017-11-11 16:56:41,092 INFO consumer-0 20
2017-11-11 16:56:42,093 INFO producer 4
2017-11-11 16:56:42,095 INFO consumer-1 4
2017-11-11 16:56:42,095 INFO consumer-2 4
2017-11-11 16:56:42,096 

In [165]:
cond = threading.Condition()

In [166]:
help(cond.notify_all)

Help on method notify_all in module threading:

notify_all() method of threading.Condition instance
    Wake up all threads waiting on this condition.
    
    If the calling thread has not acquired the lock when this method
    is called, a RuntimeError is raised.



In [167]:
help(cond.notify)

Help on method notify in module threading:

notify(n=1) method of threading.Condition instance
    Wake up one or more threads waiting on this condition, if any.
    
    If the calling thread has not acquired the lock when this method is
    called, a RuntimeError is raised.
    
    This method wakes up at most n of the threads waiting for the condition
    variable; it is a no-op if no threads are waiting.



In [168]:
class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        self.cond = threading.Condition()
        
    def consumer(self):
        while not self.event.is_set():
            with self.cond:
                self.cond.wait()
                logging.info(self.data)
        
    def producer(self):
        for _ in range(10):
            data = random.randint(0, 100)
            logging.info(data)
            self.data = data
            with self.cond:
                self.cond.notify(2)
            self.event.wait(1)
        self.event.set()

In [169]:
d = Dispatcher()
p = threading.Thread(target=d.producer, name='producer')
c = threading.Thread(target=d.consumer, name='consumer')

In [170]:
for x in range(4):
    threading.Thread(target=d.consumer, name='consumer-{}'.format(x)).start()

In [171]:
p.start()

2017-11-11 17:01:41,745 INFO producer 64
2017-11-11 17:01:41,747 INFO consumer-1 64
2017-11-11 17:01:41,749 INFO consumer-0 64
2017-11-11 17:01:42,750 INFO producer 11
2017-11-11 17:01:42,752 INFO consumer-3 11
2017-11-11 17:01:42,753 INFO consumer-2 11
2017-11-11 17:01:43,756 INFO producer 19
2017-11-11 17:01:43,757 INFO consumer-1 19
2017-11-11 17:01:43,758 INFO consumer-0 19
2017-11-11 17:01:44,757 INFO producer 13
2017-11-11 17:01:44,758 INFO consumer-2 13
2017-11-11 17:01:44,759 INFO consumer-3 13
2017-11-11 17:01:45,762 INFO producer 40
2017-11-11 17:01:45,764 INFO consumer-0 40
2017-11-11 17:01:45,766 INFO consumer-1 40
2017-11-11 17:01:46,767 INFO producer 43
2017-11-11 17:01:46,768 INFO consumer-2 43
2017-11-11 17:01:46,769 INFO consumer-3 43
2017-11-11 17:01:47,772 INFO producer 10
2017-11-11 17:01:47,773 INFO consumer-0 10
2017-11-11 17:01:47,774 INFO consumer-1 10
2017-11-11 17:01:48,776 INFO producer 67
2017-11-11 17:01:48,777 INFO consumer-2 67
2017-11-11 17:01:48,778 INF

- event是用来 阻塞的
- condition 是用来 做 生产者消费者模型

In [172]:
cond.acquire()

True

In [173]:
cond.release()

- event 用于 线程之间的 事件通知
- lock 用于 保护共享资源
- condition用于生产者消费者模型

- 信号量
- 栅栏